In [ ]:
def plot_loss_curves(history):
    
    train_loss=history.history['loss']
    val_loss=history.history['val_loss']

    train_accuracy=history.history['acc']
    val_accuracy=history.history['val_acc']

    epochs=range(1,len(history.history['loss'])+1)
    plt.figure(figsize=(20,7))
    
  # plot loss 
    plt.subplot(1,2,1)
    plt.plot(epochs,train_loss,label="training_loss")
    plt.plot(epochs,val_loss,label="validation_loss")
    plt.title("Loss curves")
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend()

  # plot accuracy 
    plt.subplot(1,2,2)
    plt.plot(epochs,train_accuracy,label="training_acc")
    plt.plot(epochs,val_accuracy,label="validation_acc")
    plt.title("Accuracy curves")
    plt.xlabel('epochs')
    plt.ylabel('Accuracy')
    plt.legend()

In [ ]:
def evaluation_results(results):
    for name ,result in results.items():
        print('Model Name: ',name,'\n')
        print('Loss: {} \tAccuracy: {}\n'.format(result[0], result[1]))

evaluation_results_collection = {}

### 1. Import modules

In [ ]:
import os, shutil
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model

### 2. Create File Structure and import data

In [ ]:
#after you saved the folder in your current directory, specify its path:
original_dataset_dir = "./data"

original_train_dir = os.path.join(original_dataset_dir, 'Training')

In [ ]:
# rename the images to the given categories
labels_train = pd.read_csv('data/labels_training.csv', sep = ';')

mapping = dict(zip(labels_train['name'], labels_train['category']))

for i, filename in enumerate(os.listdir(original_train_dir)):
    
    if filename in mapping:
        category = mapping[filename]
        new_filename = category + '.' + str(i) + '.' + filename.split('.')[-1]
        os.rename(os.path.join(original_train_dir, filename), os.path.join(original_train_dir, new_filename))

In [ ]:
# get label Name   
def get_Label(number):
    labels = {0:'bralette', 1:'fullcup', 2: 'balcony', 3: 'plunge'}
    return labels[number]


# 0 for cat ,1 for dog
all_categories = []
images = os.listdir(original_train_dir)
for file in images:
    if file.split('.')[0] == 'bralette':
        all_categories.append(0)
    elif file.split('.')[0] == 'fullcup':
        all_categories.append(1)
    elif file.split('.')[0] == 'balcony':
        all_categories.append(2)
    else:
        all_categories.append(3)
#creat data fram to save each image with its label 
df = pd.DataFrame({
    'image': images,
    'category':all_categories})

df.head(5)

In [ ]:
print("Label-File: "  + str(len(df.loc[df['category'] == 0])) +"\t" + " Mapped images: " + str(len(labels_train.loc[labels_train['category'] == 'bralette'])))
print("Label-File: "  + str(len(df.loc[df['category'] == 1])) +"\t" + " Mapped images: " + str(len(labels_train.loc[labels_train['category'] == 'fullcup'])))
print("Label-File: "  + str(len(df.loc[df['category'] == 2])) +"\t" + " Mapped images: " + str(len(labels_train.loc[labels_train['category'] == 'balcony'])))
print("Label-File: "  + str(len(df.loc[df['category'] == 3])) +"\t" + " Mapped images: " + str(len(labels_train.loc[labels_train['category'] == 'plunge'])))

In [ ]:
#This is where we will store the images that have been split:
base_dir = "./bra_split"

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
validate_dir = os.path.join(base_dir, 'validate')


for directory in [base_dir, train_dir, test_dir, validate_dir]:
    
    if not os.path.exists(directory):

        os.mkdir(directory)
        print(directory, "created")
        

subdirectories_bralette = [] 
subdirectories_fullcup = []
subdirectories_balcony = []
subdirectories_plunge = [] 


for subdirectory in [train_dir, test_dir, validate_dir]:
    subdirectories_bralette.append(os.path.join(subdirectory, 'bralette'))
    if not os.path.exists(os.path.join(subdirectory, 'bralette')):
        os.mkdir(os.path.join(subdirectory, 'bralette'))
        print(os.path.join(subdirectory, 'bralette'), "created")
        
    subdirectories_fullcup.append(os.path.join(subdirectory, 'fullcup'))
    if not os.path.exists(os.path.join(subdirectory, 'fullcup')):
        os.mkdir(os.path.join(subdirectory, 'fullcup'))
        print(os.path.join(subdirectory, 'fullcup'), "created")

    subdirectories_balcony.append(os.path.join(subdirectory, 'balcony'))
    if not os.path.exists(os.path.join(subdirectory, 'balcony')):
        os.mkdir(os.path.join(subdirectory, 'balcony'))
        print(os.path.join(subdirectory, 'balcony'), "created")
    
    subdirectories_plunge.append(os.path.join(subdirectory, 'plunge'))
    if not os.path.exists(os.path.join(subdirectory, 'plunge')):
        os.mkdir(os.path.join(subdirectory, 'plunge'))
        print(os.path.join(subdirectory, 'plunge'), "created")

In [ ]:
images = os.listdir(original_train_dir)
# taking 80% as train data 
x_train, x_test1, y_train, y_test1 = train_test_split(images, df["category"] ,test_size=0.2, random_state=42)

# from 20% data, splitting 50% as test data and 50% as validation data
x_test, x_val, y_test, y_val = train_test_split(x_test1, y_test1 ,test_size=0.5, random_state=42)

print('Training data set size :', y_train.shape[0])
print('Test data set size :', y_test.shape[0])
print('Validation data set size :', y_val.shape[0])

In [ ]:
#alternative split

# images = os.listdir(original_train_dir)
# # taking 90% as train data 
# x_train, x_test1, y_train, y_test1 = train_test_split(images, df["category"] ,test_size=0.1, random_state=42)

# # from 10% data, splitting 5% as test data and 95% as validation data
# x_test, x_val, y_test, y_val = train_test_split(x_test1, y_test1 ,test_size=0.95, random_state=42)

# print('Training data set size :', y_train.shape[0])
# print('Test data set size :', y_test.shape[0])
# print('Validation data set size :', y_val.shape[0])

In [ ]:
for ix, images in enumerate([x_train, x_test, x_val]):
    for file in images:
        if file.split('.')[0] == 'bralette':
            src = os.path.join(original_train_dir, file)
            dst = os.path.join(subdirectories_bralette[ix], file)
            shutil.copyfile(src, dst)
        
        elif file.split('.')[0] == 'fullcup':
            src = os.path.join(original_train_dir, file)
            dst = os.path.join(subdirectories_fullcup[ix], file)
            shutil.copyfile(src, dst)
        
        elif file.split('.')[0] == 'balcony':
            src = os.path.join(original_train_dir, file)
            dst = os.path.join(subdirectories_balcony[ix], file)
            shutil.copyfile(src, dst)

        else:
            src = os.path.join(original_train_dir, file)
            dst = os.path.join(subdirectories_plunge[ix], file)
            shutil.copyfile(src, dst)

    print(len(os.listdir(subdirectories_bralette[ix])), " bralette copied to:", subdirectories_bralette[ix])
    print(len(os.listdir(subdirectories_fullcup[ix])), " fullcup copied to:", subdirectories_fullcup[ix])
    print(len(os.listdir(subdirectories_balcony[ix])), " balcony copied to:", subdirectories_balcony[ix])
    print(len(os.listdir(subdirectories_plunge[ix])), " plunge copied to:", subdirectories_plunge[ix])

In [ ]:
plt.figure(figsize=(20,15))
plt.suptitle("Train Images Bralette", fontsize=20)
counter =0

for i,img in enumerate(os.listdir(subdirectories_bralette[0]))  :
        plt.subplot(5,5,i+1)

        full_image= plt.imread(os.path.join(original_train_dir,img))
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(full_image, cmap=plt.cm.binary) 
        if i == 24:
            break

In [ ]:
plt.figure(figsize=(20,15))
plt.suptitle("Train Images Balcony", fontsize=20)
counter =0

for i,img in enumerate(os.listdir(subdirectories_balcony[0]))  :
        plt.subplot(5,5,i+1)

        full_image= plt.imread(os.path.join(original_train_dir,img))
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(full_image, cmap=plt.cm.binary) 
        if i == 24:
            break

In [ ]:
plt.figure(figsize=(20,15))
plt.suptitle("Train Images Fullcup", fontsize=20)
counter =0

for i,img in enumerate(os.listdir(subdirectories_fullcup[0]))  :
        plt.subplot(5,5,i+1)

        full_image= plt.imread(os.path.join(original_train_dir,img))
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(full_image, cmap=plt.cm.binary) 
        if i == 24:
            break

In [ ]:
plt.figure(figsize=(20,15))
plt.suptitle("Train Images Plunge", fontsize=20)
counter =0

for i,img in enumerate(os.listdir(subdirectories_plunge[0]))  :
        plt.subplot(5,5,i+1)

        full_image= plt.imread(os.path.join(original_train_dir,img))
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(full_image, cmap=plt.cm.binary)
        if i == 24:
            break 

### 3. Data Sampling

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,     # sampling size in each group 
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
        class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(150, 150),
        batch_size=20,
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
        class_mode='categorical')

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

### 4. Models

FCNN

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Reshape

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=25,
        color_mode='rgb',
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=25,
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

network = models.Sequential()
reshape_layer = Reshape((150 * 150 * 3,))
network.add(reshape_layer)
network.add(layers.Dense(128, activation='relu', input_shape=(150 * 150 * 3,)))
network.add(layers.Dense(256, activation='relu'))
network.add(layers.Dense(512, activation='relu'))
network.add(layers.Dense(128, activation='relu'))
network.add(layers.Dense(4, activation='softmax'))


network.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0003),
              metrics=['acc'])


# number of samples in the training and validation sets
train_samples = len(train_generator.filenames)
val_samples = len(validation_generator.filenames)

# Set the batch size
batch_size = 25

# Calculation of the the steps per epoch and validation steps
steps_per_epoch = train_samples // batch_size
validation_steps = val_samples // batch_size


# Train the model
history_network = network.fit(
  train_generator,
  steps_per_epoch=steps_per_epoch,      
  epochs=30,
  validation_data=validation_generator,
  validation_steps=validation_steps,
)

In [ ]:
plot_loss_curves(history_network)

In [ ]:
# Load the test data using the test_generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
    class_mode='categorical')

# Evaluate the model on the test data
test_loss, test_acc = network.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)

In [ ]:
#Save results

model_result = (test_loss, test_acc)

evaluation_results_collection.update({"network_model":model_result}) 

evaluation_results(evaluation_results_collection)

Basic CNN - Model 1

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

test_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,     # sampling size in each group 
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
        class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(150, 150),
        batch_size=20,
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
        class_mode='categorical')


basic_model = models.Sequential()
basic_model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
basic_model.add(layers.MaxPooling2D((2, 2)))
basic_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
basic_model.add(layers.MaxPooling2D((2, 2)))
basic_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
basic_model.add(layers.MaxPooling2D((2, 2)))
basic_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
basic_model.add(layers.MaxPooling2D((2, 2)))
basic_model.add(layers.Flatten())
basic_model.add(layers.Dense(512, activation='relu'))
basic_model.add(layers.Dropout(0.1))
basic_model.add(layers.Dense(4, activation='softmax'))  
   
basic_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])


# number of samples in the training and validation sets
train_samples = len(train_generator.filenames)
val_samples = len(validation_generator.filenames)

# Set the batch size
batch_size = 20

# Calculation of the the steps per epoch and validation steps
steps_per_epoch = train_samples // batch_size
validation_steps = val_samples // batch_size

#Train the model
history_basic_model = basic_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_steps,
)

In [ ]:
plot_loss_curves(history_basic_model)

In [ ]:
# Load the test data using the test_generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
    class_mode='categorical')

# Evaluate the model on the test data
test_loss, test_acc = basic_model.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)

In [ ]:
#Save results

model_result = (test_loss, test_acc)

evaluation_results_collection.update({"basic_cnn_model":model_result}) 

evaluation_results(evaluation_results_collection)

Frozen VGG16 - Model 2

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top = False, input_shape = (150,150,3))

Frozen_model = models.Sequential()
Frozen_model.add(conv_base)
Frozen_model.add(layers.Flatten())
Frozen_model.add(layers.Dense(1024, activation='relu'))
Frozen_model.add(layers.Dense(512, activation='relu'))
Frozen_model.add(layers.Dropout(0.5))
Frozen_model.add(layers.Dense(4, activation='softmax'))

conv_base.trainable = False


train_datagen = ImageDataGenerator(
      rescale=1./255,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      fill_mode='nearest'
      )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

Frozen_model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=2e-5),
              metrics=['acc'])


# number of samples in the training and validation sets
train_samples = len(train_generator.filenames)
val_samples = len(validation_generator.filenames)

# Set the batch size
batch_size = 20

# Calculation of the the steps per epoch and validation steps
steps_per_epoch = train_samples // batch_size
validation_steps = val_samples // batch_size

# Train the model
history_Frozen_model = Frozen_model.fit(
  train_generator,
  steps_per_epoch=steps_per_epoch,      
  epochs=30,
  validation_data=validation_generator,
  validation_steps=validation_steps,
)

In [ ]:
plot_loss_curves(history_Frozen_model)

In [ ]:
# Load the test data using the test_generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
    class_mode='categorical')

# Evaluate the model on the test data
test_loss, test_acc = Frozen_model.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)

In [ ]:
#Save results

Frozen_model_result = (test_loss, test_acc)

evaluation_results_collection.update({"Frozen_model":Frozen_model_result}) 

evaluation_results(evaluation_results_collection)

Unfrozen VGG16 - Model 3 (Best Model)

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

conv_base = VGG16(weights='imagenet', include_top = False, input_shape = (180,180,3))

VGG16_model = models.Sequential()
VGG16_model.add(conv_base)
VGG16_model.add(layers.Flatten())
VGG16_model.add(layers.Dense(64, activation='relu'))
VGG16_model.add(layers.Dense(4, activation='softmax'))

train_datagen = ImageDataGenerator(
      rescale=1./255,
      zoom_range=[0.9,1.1],
      brightness_range = [0.8, 1.2],
      fill_mode='nearest'
      )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(180, 180),
        batch_size=25,
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(180, 180),
        batch_size=25,
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

VGG16_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.00001),
              metrics=['acc'])


# number of samples in the training and validation sets
train_samples = len(train_generator.filenames)
val_samples = len(validation_generator.filenames)

# Set the batch size
batch_size = 25

# Calculation of the the steps per epoch and validation steps
steps_per_epoch = train_samples // batch_size
validation_steps = val_samples // batch_size

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the callbacks
early_stop = EarlyStopping(monitor='val_acc', patience=7)
model_checkpoint = ModelCheckpoint('best_model.h5', 
                                    save_best_only=True, 
                                    monitor='val_acc', 
                                    mode='max', 
                                    verbose=1)

# Train the model
history_VGG16 = VGG16_model.fit(
  train_generator,
  steps_per_epoch=steps_per_epoch,      
  epochs=30,
  validation_data=validation_generator,
  validation_steps=validation_steps,
  callbacks=[early_stop, model_checkpoint]
)

In [ ]:
plot_loss_curves(history_VGG16)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('best_model.h5')


# Load the test data using the test_generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(180, 180),
    batch_size=25,
    classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
    class_mode='categorical')

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)

In [ ]:
#Save results

Model_result = (test_loss, test_acc)

evaluation_results_colle\ction.update({"Unfrozen_model":Model_result}) 

evaluation_results(evaluation_results_collection)

VGG19

In [ ]:
from tensorflow.keras.applications import VGG19

conv_base = VGG19(weights='imagenet', include_top = False, input_shape = (150,150,3))

VGG19_model = models.Sequential()
VGG19_model.add(conv_base)
VGG19_model.add(layers.Flatten())
VGG19_model.add(layers.Dense(64, activation='relu'))
VGG19_model.add(layers.Dense(4, activation='softmax'))

train_datagen = ImageDataGenerator(
      rescale=1./255,
      zoom_range=[0.9,1.1],
      brightness_range = [0.8, 1.2],
      fill_mode='nearest'
      )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

VGG19_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.00001),
              metrics=['acc'])


# number of samples in the training and validation sets
train_samples = len(train_generator.filenames)
val_samples = len(validation_generator.filenames)

# Set the batch size
batch_size = 20

# Calculation of the the steps per epoch and validation steps
steps_per_epoch = train_samples // batch_size
validation_steps = val_samples // batch_size

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the callbacks
early_stop = EarlyStopping(monitor='val_acc', patience=7)
model_checkpoint = ModelCheckpoint('best_VGG19.h5', 
                                    save_best_only=True, 
                                    monitor='val_acc', 
                                    mode='max', 
                                    verbose=1)

# Train the model
history_VGG19 = VGG19_model.fit(
  train_generator,
  steps_per_epoch=steps_per_epoch,      
  epochs=30,
  validation_data=validation_generator,
  validation_steps=validation_steps,
  callbacks=[early_stop, model_checkpoint]
)

In [ ]:
plot_loss_curves(history_VGG19)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('best_VGG19.h5')


# Load the test data using the test_generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
    class_mode='categorical')

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)

In [ ]:
#Save results

Model_result = (test_loss, test_acc)

evaluation_results_collection.update({"VGG19_model":Model_result}) 

evaluation_results(evaluation_results_collection)

ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50

conv_base = ResNet50(weights='imagenet', include_top = False, input_shape = (150,150,3))

ResNet50_model = models.Sequential()
ResNet50_model.add(conv_base)
ResNet50_model.add(layers.Flatten())
ResNet50_model.add(layers.Dense(64, activation='relu'))
ResNet50_model.add(layers.Dense(4, activation='softmax'))

train_datagen = ImageDataGenerator(
      rescale=1./255,
      zoom_range=[0.9,1.1],
      brightness_range = [0.8, 1.2],
      fill_mode='nearest'
      )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical',
        classes = ['bralette', 'fullcup', 'balcony', 'plunge'])

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

ResNet50_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.00001),
              metrics=['acc'])


# number of samples in the training and validation sets
train_samples = len(train_generator.filenames)
val_samples = len(validation_generator.filenames)

# Set the batch size
batch_size = 20

# Calculation of the the steps per epoch and validation steps
steps_per_epoch = train_samples // batch_size
validation_steps = val_samples // batch_size

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the callbacks
early_stop = EarlyStopping(monitor='val_acc', patience=7)
model_checkpoint = ModelCheckpoint('best_ResNet50.h5', 
                                    save_best_only=True, 
                                    monitor='val_acc', 
                                    mode='max', 
                                    verbose=1)

# Train the model
history_ResNet50 = ResNet50_model.fit(
  train_generator,
  steps_per_epoch=steps_per_epoch,      
  epochs=30,
  validation_data=validation_generator,
  validation_steps=validation_steps,
  callbacks=[early_stop, model_checkpoint]
)

In [ ]:
plot_loss_curves(history_ResNet50)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('best_resnet50.h5')


# Load the test data using the test_generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    classes = ['bralette', 'fullcup', 'balcony', 'plunge'],
    class_mode='categorical')

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)

In [ ]:
#Save results

Model_result = (test_loss, test_acc)

evaluation_results_collection.update({"ResNet50_model":Model_result}) 

evaluation_results(evaluation_results_collection)

### 5. Prediction & Submission

In [ ]:
img_size = (180, 180)

img_dir = "./data/Testing"


class_names = {
    0: 'bralette',
    1: 'fullcup',
    2: 'balcony',
    3: 'plunge'
}

predict_results=[]

from tensorflow.keras.preprocessing import image
import numpy as np

for img_file in os.listdir(img_dir):
    # Load the image and preprocess it
    
    img_path = os.path.join(img_dir, img_file)
    img = image.load_img(img_path, target_size=img_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img/255, axis=0)

    # Make predictions
    predictions = model.predict(img)
    
    class_idx = np.argmax(predictions[0])
    class_prob = predictions[0][class_idx]

    class_name = class_names[class_idx]
    print(f'{img_file}: {class_name} ({class_prob})')
    predict_results.append((img_file, class_name))

In [ ]:
import csv
# Define the file name and headers for the CSV file
file_name = 'predictions_final.csv'
field_names = ['id', 'label']

# Write the predictions to the CSV file
with open(file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(field_names)
    for prediction in predict_results:
        writer.writerow(prediction)